In [37]:
from pymc3 import Continuous
import theano.tensor as T
from theano.tensor.nlinalg import eig, matrix_inverse

class DiscreteObsMJP(Continuous):

    def __init__(self, Q, *args, **kwargs):
        super(DiscreteObsMJP, self).__init__(*args, **kwargs)
        self.Q = Q

    def logp(self, value):
        Q = self.Q
        step_sizes = value[0]

        l = 0.0
        for tau in step_sizes:
            #get P(tau)
            eigen = eig(Q)
            lambdas = eigen[0]
            U = eigen[1]
            
            P = U.dot(T.diag(T.exp(tau*lambdas))).dot(matrix_inverse(U))
        
            #compute likelihood in terms of P(tau)
            l += T.sum(value[1][step_sizes]*T.log(P))
            
        return l

In [38]:
import rpy2.robjects as robjects
import numpy as np

robjects.r['load']("../data/metzner07_example1_sim.Rdata")
C=np.array(robjects.r['C'])
C=np.transpose(np.array(C))
for i in range(0, 2):
    C[i] = np.transpose(C[i])
    
step_sizes = np.array([0.01, 1, 100])
    
data=(step_sizes, C)

In [42]:
from pymc3 import Model, HalfNormal, Poisson
from theano.compile.ops import as_op

@as_op(itypes=[T.cmatrix], otypes=[T.cmatrix])
def convertFromRaw(Q_raw):
    nrows = Q_raw.shape[0]
    ncols = Q_raw.shape[1]
    Q = np.empty([nrows,ncols+1], dtype=np.complex64)
    for i in range(0, nrows):
        index = 0
        for j in range(0, ncols+1):
            if i==j:
                Q[i,j] = np.complex64(-sum(Q_raw[i,]))
            else:
                Q[i,j] = Q_raw[i, index]
                index = index + 1
    return Q

with Model() as model:
    '''
    Q11 = HalfNormal('Q11', sd=20)
    Q12 = HalfNormal('Q12', sd=20)
    Q13 = HalfNormal('Q13', sd=20)
    Q14 = HalfNormal('Q14', sd=20)
    
    Q21 = HalfNormal('Q21', sd=20)
    Q22 = HalfNormal('Q22', sd=20)
    Q23 = HalfNormal('Q23', sd=20)
    Q24 = HalfNormal('Q24', sd=20)
    
    Q31 = HalfNormal('Q31', sd=20)
    Q32 = HalfNormal('Q32', sd=20)
    Q33 = HalfNormal('Q33', sd=20)
    Q34 = HalfNormal('Q34', sd=20)
    
    Q41 = HalfNormal('Q41', sd=20)
    Q42 = HalfNormal('Q42', sd=20)
    Q43 = HalfNormal('Q43', sd=20)
    Q44 = HalfNormal('Q44', sd=20)
    
    Q51 = HalfNormal('Q51', sd=20)
    Q52 = HalfNormal('Q52', sd=20)
    Q53 = HalfNormal('Q53', sd=20)
    Q54 = HalfNormal('Q54', sd=20)
    '''
    Q_raw = Poisson('Q_raw', 1)
    
    Q = convertFromRaw(Q_raw)
    
    C_obs = DiscreteObsMJP('C_obs', Q=Q, step_sizes=step_sizes, observed=C)

AssertionError: 

In [ ]:
Q=[[-6, 2, 2, 1, 1], 
   [1, -4, 0, 1, 2],
   [1, 0, -4, 2, 1],
   [2, 1, 0, -3, 0],
   [1, 1, 1, 1, -4]]
Q_raw = np.array([[2, 2, 1, 1], 
   [1, 0, 1, 2],
   [1, 0, 2, 1],
   [2, 1, 0, 0],
   [1, 1, 1, 1]], dtype=np.complex64)
